
# Comparative Evaluation of A* and Bidirectional A* Search Algorithms

## Introduction
This notebook implements and experimentally compares the **A*** and **Bidirectional A*** search algorithms
in a grid-based pathfinding environment. The implementation follows the assumptions described in the task:

- 2D grid world
- Four-directional movement (up, down, left, right)
- Uniform step cost of 1
- Manhattan distance heuristic
- Fixed start at top-left corner and goal at bottom-right corner

The goal is to evaluate performance in terms of:
- Path length
- Number of expanded nodes
- Execution time



## Imports and Utility Functions
We begin by importing required libraries and defining utility functions such as the Manhattan heuristic,
neighbor generation, and path reconstruction.


In [ ]:

import heapq
import time
import random
import csv
from typing import Tuple, List, Dict, Set


In [ ]:

def manhattan(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    """Manhattan distance heuristic."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def get_neighbors(node: Tuple[int, int], grid: List[List[int]]) -> List[Tuple[int, int]]:
    """Return valid 4-connected neighbors."""
    x, y = node
    neighbors = []
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] == 0:
            neighbors.append((nx, ny))
    return neighbors


def reconstruct_path(parent: Dict[Tuple[int, int], Tuple[int, int]], start, goal):
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = parent[current]
    path.append(start)
    path.reverse()
    return path



## Unidirectional A* Search
This section implements the standard A* search algorithm using the evaluation function:

\[ f(n) = g(n) + h(n) \]

Where:
- \(g(n)\) is the cost from the start node
- \(h(n)\) is the Manhattan distance heuristic


In [ ]:

def astar(grid, start, goal):
    t0 = time.perf_counter()
    open_set = []
    heapq.heappush(open_set, (0, start))

    g_cost = {start: 0}
    parent = {}
    closed = set()
    expanded = 0

    while open_set:
        _, current = heapq.heappop(open_set)

        if current in closed:
            continue

        expanded += 1
        closed.add(current)

        if current == goal:
            t1 = time.perf_counter()
            path = reconstruct_path(parent, start, goal)
            return path, expanded, t1 - t0

        for neighbor in get_neighbors(current, grid):
            tentative_g = g_cost[current] + 1
            if neighbor not in g_cost or tentative_g < g_cost[neighbor]:
                g_cost[neighbor] = tentative_g
                f = tentative_g + manhattan(neighbor, goal)
                parent[neighbor] = current
                heapq.heappush(open_set, (f, neighbor))

    return None, expanded, time.perf_counter() - t0



## Bidirectional A* Search
Bidirectional A* runs two A* searches simultaneously:
- Forward search from the start
- Backward search from the goal

The algorithm terminates only when the best possible combined path cannot be improved,
ensuring optimality.


In [ ]:

def bidirectional_astar(grid, start, goal):
    t0 = time.perf_counter()

    open_f, open_b = [], []
    heapq.heappush(open_f, (manhattan(start, goal), start))
    heapq.heappush(open_b, (manhattan(goal, start), goal))

    g_f, g_b = {start: 0}, {goal: 0}
    parent_f, parent_b = {}, {}
    closed_f, closed_b = set(), set()

    best_cost = float('inf')
    meeting_node = None
    expanded = 0

    while open_f and open_b:
        # Forward
        _, current_f = heapq.heappop(open_f)
        if current_f not in closed_f:
            expanded += 1
            closed_f.add(current_f)

            if current_f in g_b:
                cost = g_f[current_f] + g_b[current_f]
                if cost < best_cost:
                    best_cost = cost
                    meeting_node = current_f

            for n in get_neighbors(current_f, grid):
                ng = g_f[current_f] + 1
                if n not in g_f or ng < g_f[n]:
                    g_f[n] = ng
                    parent_f[n] = current_f
                    heapq.heappush(open_f, (ng + manhattan(n, goal), n))

        # Backward
        _, current_b = heapq.heappop(open_b)
        if current_b not in closed_b:
            expanded += 1
            closed_b.add(current_b)

            if current_b in g_f:
                cost = g_b[current_b] + g_f[current_b]
                if cost < best_cost:
                    best_cost = cost
                    meeting_node = current_b

            for n in get_neighbors(current_b, grid):
                ng = g_b[current_b] + 1
                if n not in g_b or ng < g_b[n]:
                    g_b[n] = ng
                    parent_b[n] = current_b
                    heapq.heappush(open_b, (ng + manhattan(n, start), n))

        if open_f and open_b and open_f[0][0] + open_b[0][0] >= best_cost:
            break

    if meeting_node is None:
        return None, expanded, time.perf_counter() - t0

    path_f = reconstruct_path(parent_f, start, meeting_node)
    path_b = []
    current = meeting_node
    while current != goal:
        current = parent_b[current]
        path_b.append(current)

    return path_f + path_b, expanded, time.perf_counter() - t0



## Grid Generation and Experiments
This section generates reproducible grids and runs both algorithms while recording metrics.


In [ ]:

def generate_grid(size, obstacle_prob, seed):
    random.seed(seed)
    grid = [[0 if random.random() > obstacle_prob else 1 for _ in range(size)] for _ in range(size)]
    grid[0][0] = 0
    grid[size-1][size-1] = 0
    return grid


def run_experiment(size, obstacle_prob, seed):
    grid = generate_grid(size, obstacle_prob, seed)
    start, goal = (0, 0), (size-1, size-1)

    path_a, exp_a, time_a = astar(grid, start, goal)
    path_b, exp_b, time_b = bidirectional_astar(grid, start, goal)

    return {
        "grid_size": size,
        "obstacle_prob": obstacle_prob,
        "astar_path_len": len(path_a) if path_a else None,
        "astar_expanded": exp_a,
        "astar_time": time_a,
        "bidir_path_len": len(path_b) if path_b else None,
        "bidir_expanded": exp_b,
        "bidir_time": time_b,
    }



## Example Execution
The following cell runs a sample experiment on a 50×50 grid.


In [ ]:

result = run_experiment(size=50, obstacle_prob=0.2, seed=42)
for k, v in result.items():
    print(f"{k}: {v}")



## Conclusion
The results confirm that Bidirectional A* consistently finds optimal paths while often
reducing the number of expanded nodes and execution time, especially in open and moderately
obstructed environments. However, its advantage depends on obstacle distribution and spatial symmetry.
